In [ ]:
from scipy import signal
import matplotlib.pyplot as plt
import numpy as np
import collections
import math

In [ ]:
sample_rate = 1e4
duration = 0.04
tone_freq = 2e3
samples_to_plot = 100

In [ ]:
class Waveform:
    def __init__(self, t0, dt, y):
        self.t0 = t0
        self.dt = dt
        self.y = y

    def sample_times(self):
        begin = self.t0
        end = self.t0 + self.dt * len(self.y)
        return np.arange(begin, end, self.dt)
    
    def __add__(self, other):
        return Waveform(self.t0, self.dt, self.y + other.y)
    
class Spectrum:
    def __init__(self, f0, df, y):
        self.f0 = f0
        self.df = df
        self.y = y
        
    def frequency_bins(self):
        begin = self.f0
        end = self.f0 + self.df * len(self.y)
        return np.arange(begin, end, self.df)

In [ ]:
def create_single_tone(sample_rate, duration, tone_freq, t0=0):
    dt = 1/sample_rate
    t = np.arange(t0, t0+duration, dt)
    tone = np.exp(1.0j*(2*np.pi*tone_freq*t))
    return Waveform(t0, dt, tone)

In [ ]:
# Check phase continuity
tone = create_single_tone(1e3, 1, 2.5)
plt.plot(tone.sample_times(), np.real(tone.y))
tone = create_single_tone(1e3, 1, 2.5, 1)
plt.plot(tone.sample_times(), np.real(tone.y))

In [ ]:
tone = create_single_tone(sample_rate, duration, tone_freq)
time_axis = tone.sample_times()
plt.plot(time_axis[:samples_to_plot], np.real(tone.y)[:samples_to_plot])
plt.plot(time_axis[:samples_to_plot], np.imag(tone.y)[:samples_to_plot])

In [ ]:
def create_noise_signal(sample_rate, duration, amplitude, t0=0):
    real = np.random.normal(0, amplitude, int(round(sample_rate*duration)))
    cplx = np.random.normal(0, amplitude, int(round(sample_rate*duration)))
    return Waveform(t0, 1.0/sample_rate, real + 1.0j*cplx)

In [ ]:
noise = create_noise_signal(sample_rate, duration, 0.01)

In [ ]:
plt.plot(time_axis[:samples_to_plot], np.real(noise.y)[:samples_to_plot])
plt.plot(time_axis[:samples_to_plot], np.imag(noise.y)[:samples_to_plot])

In [ ]:
wfm = tone + noise
plt.plot(time_axis[:samples_to_plot], np.real(wfm.y)[:samples_to_plot])
plt.plot(time_axis[:samples_to_plot], np.imag(wfm.y)[:samples_to_plot])

In [ ]:
def windowed_fft(wfm, comp_for_demod_loss=True):
    num_samps = len(wfm.y)
    
    # Apply window
    window = np.hamming(num_samps)
    y_windowed = np.multiply(window, wfm.y)
    
    # Calculate FFT and shift it so frequency bins are in order
    fft = np.fft.fftshift(np.fft.fft(y_windowed, len(y_windowed)))

    # Scale the fft by the number of elements and by the window. Also scale
    # by sqrt(2) to compensate for loss of half of the signal energy when
    # low-pass filtering during IQ demodulation.
    window_coherent_gain = 0.54
    scaling = 1.0 / (len(fft) * window_coherent_gain)
    if comp_for_demod_loss:
        scaling *= np.sqrt(2)
    fft *= scaling
    
    # Calculate the bin information
    df = (1.0/wfm.dt) / num_samps
    f0 = -(num_samps / 2.0) * df
    return Spectrum(f0, df, fft)

def convert_spectrum_to_volts_squared(spectrum):
    v_squared = np.real(spectrum.y)**2 + np.imag(spectrum.y)**2
    return Spectrum(spectrum.f0, spectrum.df, v_squared)


spectrum = windowed_fft(wfm)
spectrum_volts_squared = convert_spectrum_to_volts_squared(spectrum)
plt.plot(spectrum_volts_squared.frequency_bins(), spectrum_volts_squared.y)

In [ ]:
def convert_spectrum_to_dbm(spectrum):
    """
    Note: assumes 50 Ohms
    """
    volts_squared = convert_spectrum_to_volts_squared(spectrum)
    dbv = 10*np.log10(volts_squared.y)
    dbm = dbv + 30 - 10*np.log10(50)
    return Spectrum(spectrum.f0, spectrum.df, dbm)

spectrum_dbm = convert_spectrum_to_dbm(spectrum)
bins = spectrum_dbm.frequency_bins()
plt.plot(bins, spectrum_dbm.y)
peak_index, _ = signal.find_peaks(spectrum_dbm.y, height=-20)
peak_freq = bins[peak_index]
peak_ampl = spectrum_dbm.y[peak_index]
print("Peak freq = {}, ampl = {}".format(peak_freq, peak_ampl))
plt.plot(peak_freq, peak_ampl, 'x')

In [ ]:
def frequency_shift(wfm, shift):
    tone = create_single_tone(1.0/wfm.dt, wfm.dt * len(wfm.y), shift)
    y = wfm.y * tone.y
    return Waveform(wfm.t0, wfm.dt, y)

tone = create_single_tone(sample_rate, duration, tone_freq)
noise = create_noise_signal(sample_rate, duration, 0.01)
wfm = tone + noise
shifted = frequency_shift(wfm, -1000)
spectrum = windowed_fft(shifted)
spectrum_dbm = convert_spectrum_to_dbm(spectrum)
plt.plot(spectrum_dbm.frequency_bins(), spectrum_dbm.y)

In [ ]:
def create_dual_tone(sample_rate, duration, tone_freq, t0=0):
    dt = 1/sample_rate
    t = np.arange(t0, t0+duration, dt)
    tone = np.sin(2*np.pi*tone_freq*t)
    return Waveform(t0, dt, tone)

In [ ]:
tone = create_dual_tone(sample_rate, duration, tone_freq)
noise = create_noise_signal(sample_rate, duration, 0.01)
wfm = tone + noise
spectrum = windowed_fft(wfm)
spectrum_dbm = convert_spectrum_to_dbm(spectrum)
plt.plot(spectrum_dbm.frequency_bins(), spectrum_dbm.y)

In [ ]:
def decimate_wfm(wfm, decim):
    new_y = signal.resample_poly(wfm.y, 1, decim)
    return Waveform(wfm.t0, wfm.dt * decim, new_y)

tone = create_dual_tone(sample_rate, duration, tone_freq)
noise = create_noise_signal(sample_rate, duration, 0.01)
wfm = tone + noise
wfm = frequency_shift(wfm, tone_freq)
wfm = decimate_wfm(wfm, 4)
spectrum = windowed_fft(wfm)
spectrum_dbm = convert_spectrum_to_dbm(spectrum)
plt.plot(spectrum_dbm.frequency_bins(), spectrum_dbm.y)

In [ ]:
def zoom_fft(wfm, center_freq, decim):
    wfm = frequency_shift(wfm, -center_freq)
    wfm = decimate_wfm(wfm, decim)
    return windowed_fft(wfm)

tone = create_dual_tone(sample_rate, duration, tone_freq)
noise = create_noise_signal(sample_rate, duration, 0.01)
wfm = tone + noise
spectrum = zoom_fft(wfm, tone_freq+1e3, 2)
spectrum_dbm = convert_spectrum_to_dbm(spectrum)
plt.plot(spectrum_dbm.frequency_bins(), spectrum_dbm.y)